#  Homework 6 for CS 247 : Advanced Data Mining Learning


### Due: 11:59 pm 05/20

##### Please read the Homework Guidance (uploaded to CCLE) carefully and make sure you fulfill all the requirements.



__Name__: [Your name]

__UID__: [Your uid]

## Problem 1: Spectral Clustering (35 pts)

In this problem, you are going to prove a property of the unormalized graph Laplacian matrix. Then, you are going to implement the spectral clustering and apply it on the Karate Club dataset.


### Part 1: Property of Unnormalized Graph Laplacian (10 pts)

Given a unnormalized graph Laplacian matrix $L=D-W$, prove that for every vector $f \in R^n$, we have:

$$
f^T L f = \frac{1}{2}\sum_{ij} w_{ij}(f_i - f_j) ^ 2
$$


#### Write Your answer here:

[Your Answer]

### Part 2: Implement Spectral Clustering (25pts = 10 + 15)

In this part, you are going to implement the spectral clustering model, and get two clusters of the Karate club graph, then visialize them. You are allowed to use the sklearn implementation of Kmeans.

Hint:
1. Please refer to the official documentation if you want to get familiar with the networkx library: https://networkx.org/documentation/stable/tutorial.html
2. You may find this link helpful if you want to know more about the Karate Club dataset: https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.generators.social.karate_club_graph.html

In [ ]:
# import neccessary libraries

import networkx as nx
from sklearn.cluster import KMeans
import numpy as np
from scipy.linalg import sqrtm 
from numpy import linalg as LA

In [ ]:
# load and preprocess the Karate Club Graph

G = nx.karate_club_graph()
nx.draw(G, with_labels=True, pos=nx.spring_layout(G))

A = nx.adj_matrix(G)
A = A.todense()

In [ ]:
def laplacian(A):
    """
        TODO: 
            Computes the symetric normalized laplacian.
                 L = D^{-1/2} A D{-1/2}
    """
    

In [ ]:
def spectral_clustering(adjacency, n_clusters):
    k_means = KMeans(n_clusters=n_clusters, random_state=43)
    """
        TODO: 
            Spectral Clustering method, please return the labels.
            You can follow these steps:
                (1) Calculate the lapalacian matrix of the adjacency matrix.
                (2) Get the eigen vector of this laplacian matrix, then normalize it.
                (3) Get the cluster label using kmeans.     
    """
    
    
    
    
    
    
    

# use the spectral clustering method on Karate Club Graph
labels = spectral_clustering(A, 2)
print(labels) # Please keep this output in your submission

In [ ]:
# visualize the results

r_nodes = []
b_nodes = []
for i, j in enumerate(labels):
    if j == 0:
        r_nodes += [i]
    else:
        b_nodes += [i]

pos = nx.spring_layout(G)
nx.draw(G,pos, with_labels=True,
                       nodelist=r_nodes,
                       node_color='r',
                       node_size=500,
                   alpha=0.8)
nx.draw(G,pos, with_labels=True,
                       nodelist=b_nodes,
                       node_color='b',
                       node_size=500,
                   alpha=0.8)

## Problem 2: Normalized Cut (35pts)

In this problem, you are going to prove some properties of the Normalized Cut.

Consider the problem of partitioning a graph G = (V,E) into two subgraphs with similar sizes. Let A ⊆ V and B ⊆ V denote the disjoint node sets of the two clusters. Let cut(A, B) denote the total number of edges or total weight of edges that separate the two clusters, i.e., cut(A,B) = $\Sigma_{i∈A,j∈B}$ wij, where wij denotes the weight associated with edge between i and j. Let dX = $\Sigma_{i∈X}$ di be the total degree of nodes in cluster X, the normalized cut between two clusters is defined as:

$Ncut(A,B)=\frac{cut(A,B)}{d_A}+\frac{cut(B,A)}{d_B}$

Let f be the cluster indicator vector, where:
$f=\begin{cases} 
\sqrt{\frac{d_B}{d_A}} & if\ i\in A\\
-\sqrt{\frac{d_A}{d_B}} & if\ i\in B
\end{cases}$

Let D be the diagonal matrix with Dii = $\Sigma_j w_{ij}$, and 0 elsewhere.

### Part 1 (5pts)

Please prove: $(Df)^T 1 = 0$

#### Write Your answer here:

[Your Answer]

### Part 2 (5pts)

Please prove: $f^TDf = d_V$ , where $d_V$ is the total degree of all the nodes in the graph.


#### Write your answer here:

[Your Answer]

### Part 3 (10pts)

Please prove: $f^TLf = d_V Ncut(A,B)$, where L is the unnormalized graph Laplacian. 

#### Write your answer here:

[Your Answer]

### Part 4 (15pts)
Prove the solution of f is the second eigenvector of $L_rw$. 

Hint: 
With part 1~3, we can relax f to a real vector, and the goal is to find f to minimize $f^TLf$, with the constraint part 1 and part 2 hold. Then, let $g = D^{1/2}f$, substitute f with g in the aforementioned objective function and constraints. Finally, prove the solution of g is the second eigenvector of Lsym.


#### Write your answer here:

[Your Answer]

## Problem 3: Node Embedding - LINE (30 pts = 20 + 10)


In this problem, you are going to implement the First-order LINE (finish contrastive loss, negative sampling and training pipleline). Get embedding of karate graph, then visualize your results.

Hint: Please refer to slide 09 - Graph and Network: Graph Embedding, page 8-12 for the details of the LINE model.

In [ ]:
# import necessary libraries

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm import trange 
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sb
import networkx as nx
import numpy as np
from scipy.linalg import sqrtm 
from numpy import linalg as LA

In [ ]:
# load dataset and set parameters

G = nx.karate_club_graph()
edges  = np.array(list(G.edges))
degree = dict(G.degree)
true_labels = np.zeros(len(G.node))
for i in range(len(labels)):
    if G.node[i]['club']=='Officer':
        true_labels[i]=1

n_epochs = 100
neg_size = 5
batchrange = 3

In [ ]:
class Line(nn.Module):
    def __init__(self, size, embed_dim=128):
        super(Line, self).__init__()

        self.embed_dim = embed_dim
        self.nodes_embeddings = nn.Embedding(size, embed_dim)

        # Initialization
        self.nodes_embeddings.weight.data = self.nodes_embeddings.weight.data.uniform_(-.5, .5) / embed_dim

    def loss(self, v_i, v_j, negsamples):
        '''
            TODO: 
                Similar to Skip-gram implementaion, implement contrastive loss here
        '''
        
        

In [ ]:
#     generating batches of data.

def makeData(samplededges, negsamplesize, degree):
    sampledNodes = set()
    nodesProb = []
    sumofDegree = 0
    for e in samplededges:
        sampledNodes.add(e[0])
        sampledNodes.add(e[1])
    sampledNodes = list(sampledNodes)
    nodesProb = [pow(degree[v],3/4) for v in sampledNodes]
    sumofDegree = sum(nodesProb)
    nodesProb[:] = [x/sumofDegree for x in nodesProb]

    for e in samplededges:
        sourcenode, targetnode = e[0], e[1]
        negnodes = []
        negsamples = 0
        while negsamples < negsamplesize:
            '''
                TODO: 
                    Randomly sampled negative nodes based on degree (d^{3/4})
            '''
            samplednode = 
            
            
            if (samplednode == sourcenode) or (samplednode == targetnode):
                continue
            else:
                negsamples += 1
                negnodes += [samplednode]
        yield [e[0], e[1]] + negnodes

        


In [ ]:
# training

line = Line(len(G), embed_dim=100)
opt = optim.Adam(line.parameters())
for epoch in range(n_epochs):
    for b in trange(batchrange):
        opt.zero_grad()
        edge_idx = np.random.choice(len(edges), 10)
        samplededges = edges[edge_idx]
        
        batch = list(makeData(samplededges, neg_size, degree))
        batch = torch.LongTensor(batch)
        
        # based on the generated batch, train LINE via minimizing the loss.
        v_i = batch[:,0]
        v_j = batch[:,1]
        negsamples =  batch[:,2:]
        loss = line.loss(v_i, v_j, negsamples)
        loss.backward()
        opt.step()


In [ ]:
# TSNE visualization, with node id on

emb  = line.nodes_embeddings.weight.data.numpy()
tsne_emb = TSNE(n_components = 2, perplexity = 5, learning_rate = 10).fit_transform(emb)

plt.scatter(tsne_emb[:,0], tsne_emb[:,1], c=true_labels)
for i in range(len(tsne_emb)):
    plt.annotate(str(i), xy=(tsne_emb[i,0], tsne_emb[i,1]))
plt.show()



In [ ]:
# heatmap visualization, check cosine similarities between all pair of nodes

res = cosine_similarity(emb) 
sb.clustermap(res)
plt.show()